In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt
import numpy as np
import PIL
from IPython import display
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings("ignore")

In [2]:
from train_DCGAN import *

In [3]:
Disc_Model = tf.keras.models.load_model('ImageNet_512/disc_weights/disc_model_epoch_0014.h5')

In [11]:
Disc_Model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 16, 16, 128)       9600      
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 256)         819200    
_________________________________________________________________
batch_normalization_10 (Batc (None, 8, 8, 256)         1024      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 512)        

In [4]:
from tensorflow.keras import layers

In [5]:
model_layers = Disc_Model.layers

output_layers = []
for layer_idx in range(len(model_layers)):
    if isinstance(model_layers[layer_idx], tf.keras.layers.LeakyReLU):
        output_layers.append(layers.Flatten()(layers.MaxPooling2D(pool_size=(2,2))(Disc_Model.layers[layer_idx].output)))

final_out = layers.Concatenate()(output_layers)
FeatureExtractor = tf.keras.models.Model(inputs=Disc_Model.input, outputs = final_out)
FeatureExtractor.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_3_input (InputLayer)     [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 16, 16, 128)  9600        conv2d_3_input[0][0]             
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 16, 16, 128)  512         conv2d_3[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_3 (LeakyReLU)       (None, 16, 16, 128)  0           batch_normalization_9[0][0]      
______________________________________________________________________________________________

In [6]:
@tf.function
def prepare_image_cifar_labeled (item):
    image = item['image']
    label = item['label']
    image = tf.cast(image, 'float32')
    image = tf.image.random_flip_left_right(image)
    image =  (image - 127.5) / 127.5 # normalize images to -1,1
    return image, label


In [7]:
cifar_10 = tfds.load('cifar10')
cifar_10_train = cifar_10['train']
cifar_10_test = cifar_10['test']

In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 512
cifar_10_train = cifar_10_train.map(prepare_image_cifar_labeled, num_parallel_calls=tf.data.experimental.AUTOTUNE)
cifar_10_train = cifar_10_train.batch(BATCH_SIZE)
cifar_10_train = cifar_10_train.prefetch(tf.data.experimental.AUTOTUNE)
cifar_10_test = cifar_10_test.map(prepare_image_cifar_labeled, num_parallel_calls=tf.data.experimental.AUTOTUNE)
cifar_10_test = cifar_10_test.batch(BATCH_SIZE)
cifar_10_test = cifar_10_test.prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
def extract_features(model, dataset):
    img_features = []
    labels = []
    for img, label in dataset:
        img_features.append(model.predict(img))
        labels.append(label)
    
    allfeatures=img_features[0]
    for i in range(len(img_features)):
        if i == 0:
            continue
        allfeatures= np.vstack((allfeatures, img_features[i]))
        
    labels = [x.numpy() for x in labels]
    labels = np.hstack(labels)
    
    return allfeatures, labels

In [10]:
train_features, train_labels = extract_features(FeatureExtractor, cifar_10_train)
test_features, test_labels = extract_features(FeatureExtractor, cifar_10_test)

In [11]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [12]:
%%time
svm_mod = OneVsRestClassifier(LinearSVC(random_state=0, tol=1e-5,verbose=1, max_iter=1000), n_jobs=5)
svm_mod.fit(train_features,train_labels)

CPU times: user 356 ms, sys: 1.11 s, total: 1.46 s
Wall time: 9min 22s


OneVsRestClassifier(estimator=LinearSVC(random_state=0, tol=1e-05, verbose=1),
                    n_jobs=5)

In [13]:
svm_mod.score(test_features,test_labels)

0.6367

In [17]:
Disc_Model = tf.keras.models.load_model('ImageNet_512_40_epoch/disc_weights/disc_model_epoch_0039.h5')

model_layers = Disc_Model.layers

output_layers = []
for layer_idx in range(len(model_layers)):
    #if isinstance(model_layers[layer_idx], tf.keras.layers.LeakyReLU):
    if isinstance(model_layers[layer_idx], tf.keras.layers.Conv2D):
        output_layers.append(layers.Flatten()(layers.MaxPooling2D(pool_size=(2,2))(Disc_Model.layers[layer_idx].output)))

final_out = layers.Concatenate()(output_layers)
FeatureExtractor = tf.keras.models.Model(inputs=Disc_Model.input, outputs = final_out)
FeatureExtractor.summary()
train_features, train_labels = extract_features(FeatureExtractor, cifar_10_train)
test_features, test_labels = extract_features(FeatureExtractor, cifar_10_test)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 128)  9600        conv2d_input[0][0]               
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 16, 16, 128)  512         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 16, 16, 128)  0           batch_normalization_3[0][0]      
____________________________________________________________________________________________

In [18]:
%%time
svm_mod = OneVsRestClassifier(LinearSVC(random_state=0, tol=1e-5,verbose=1, max_iter=1000), n_jobs=5)
svm_mod.fit(train_features,train_labels)

CPU times: user 385 ms, sys: 1.23 s, total: 1.61 s
Wall time: 10min 23s


OneVsRestClassifier(estimator=LinearSVC(random_state=0, tol=1e-05, verbose=1),
                    n_jobs=5)

In [19]:
svm_mod.score(test_features,test_labels)

0.6367